In [ ]:
!pip install -q datasets loralib sentencepiece
!pip uninstall transformers
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes
!pip install flask-ngrok flask pyngrok
!ngrok authtoken '2PGalYW6EdJxXtxCggrAixZBHd8_2V6d6q7uWYcqpmXJZHvKH'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61

In [ ]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
import textwrap


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [ ]:
tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")

model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
# model = PeftModel.from_pretrained(model, "shadow006/alpaca7B-lora-logqa-finetuned")
model = PeftModel.from_pretrained(model, "samwit/alpaca7B-lora")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
!pip install openai
import openai
openai.api_key = 'SECRET'
def alpaca_talk(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2,
    )
    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )
    resp_list = []
    for s in generation_output.sequences:
      resp = tokenizer.decode(s)
      resp = get_response_from_full_text(resp)
      resp_list.append(resp)

    # print(resp_list[0])
    return resp_list[0]

def chatgpt(prompt):
    response = openai.Completion.create(
      engine='text-davinci-003',
      prompt=prompt,
      max_tokens=256,  # Adjust the desired response length
      n=1,  # Number of responses to generate
      stop=None,  # Specify a stop sequence if desired
      temperature=0.7,  # Control the randomness of the output
      top_p=1.0,  # Control the diversity of the output
      frequency_penalty=0.0,
      presence_penalty=0.0
    )

    if 'choices' in response and len(response['choices']) > 0:
      return response['choices'][0]['text']
    else:
      return ''

# 从全文本中抓取回答
def get_response_from_full_text(full_text):
  return full_text.split("### Response:\n")[-1]


def format_prompt(instruction, input):
  prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Instruction: '''{instruction}'''
  ### Input: '''{input}'''
  ### Response:
  """
  return prompt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import locale
print(locale.getpreferredencoding())
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

UTF-8


In [ ]:
!pip install rouge
import json
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

import json
import nltk
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu_score(dataset):
    total_bleu = 0
    for qa_pair in dataset:
        question = qa_pair['instruction']
        reference_answer = qa_pair['output']
        input_code = qa_pair['input']
        final_question = format_prompt(question, input_code)
        # generated_answer = alpaca_talk(final_question)
        generated_answer = chatgpt(final_question)
        print(generated_answer)
        sb = sentence_bleu([reference_answer.split()], generated_answer.split())
        print('sentence_bleu:',sb)
        total_bleu += sb
    average_bleu = total_bleu / len(dataset)
    return average_bleu

def calculate_rouge_score(dataset):
    rouge = Rouge()
    total_rouge = { 'rouge-1': {'f': 0, 'p': 0, 'r': 0}, 'rouge-2': {'f': 0, 'p': 0, 'r': 0}, 'rouge-l': {'f': 0, 'p': 0, 'r': 0} }
    
    for qa_pair in dataset:
        question = qa_pair['instruction']
        reference_answer = qa_pair['output']
        input_code = qa_pair['input']
        final_question = format_prompt(question, input_code)
        generated_answer = alpaca_talk(final_question)
        # generated_answer = chatgpt(final_question)
        print(generated_answer)
        scores = rouge.get_scores(generated_answer, reference_answer)[0]
        print(scores)
        for key in total_rouge.keys():
            total_rouge[key]['f'] += scores[key]['f']
            total_rouge[key]['p'] += scores[key]['p']
            total_rouge[key]['r'] += scores[key]['r']
            
    average_rouge = {key: {k: v / len(dataset) for k, v in value.items()} for key, value in total_rouge.items()}
    return average_rouge

# Read the QA dataset from a JSON file
with open('/content/drive/MyDrive/5004/QA_test dataset.json', 'r') as file:
    qa_data = json.load(file)

references = []
predictions = []

# Calculate BLEU score
# bleu_score = calculate_bleu_score(qa_data)
rouge_score = calculate_rouge_score(qa_data)
print("rouge_score:", rouge_score)

# # Test Example
# instruction = """
# What are the errors in these logs?
# """
# input = """
# 2023/04/02 12:28:51 /offermaker/video_user/internal/dao/user_dao.go:88 record not found
# [5.951ms] [rows:0] SELECT * FROM `user` WHERE username = 'test3' ORDER BY `user`.`user_id` LIMIT 1
# [GIN] 2023/04/02 - 12:28:51 | 200 |  101.488053ms |  219.143.177.52 | POST     "/users/register"
# [GIN] 2023/04/02 - 13:10:41 | 404 |       2.851µs |  219.143.177.52 | GET      "/"
# [GIN] 2023/04/02 - 13:10:41 | 404 |       3.604µs |  219.143.177.52 | GET      "/favicon.ico"
# [GIN] 2023/04/02 - 13:16:37 | 204 |      49.559µs |  219.143.177.52 | OPTIONS  "/users/register"

# 2023/04/02 13:16:37 /offermaker/video_user/internal/dao/user_dao.go:83 record not found
# [6.457ms] [rows:0] SELECT * FROM `user` WHERE email = 'test0402@qq.com' ORDER BY `user`.`user_id` LIMIT 1

# """

# prompt = format_prompt(instruction, input)

# # print(prompt)
# alpaca_talk(prompt)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Generating...
  1) Check if you have imported all required packages and libraries, including "foo". If not, import them accordingly.
   2) Make sure your code has no syntax errors or typos.
{'rouge-1': {'r': 0.058823529411764705, 'p': 0.034482758620689655, 'f': 0.04347825620983037}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.058823529411764705, 'p': 0.034482758620689655, 'f': 0.04347825620983037}}
Generating...
   The most common cause for this type of runtime error is when you try to access data outside the bounds of your array or slice, such as trying to read from an element at position -100.
{'rouge-1': {'r': 0.5, 'p': 0.47058823529411764, 'f': 0.4848484798530762}, 'rouge-2': {'r': 0.2222222222222222, 'p': 0.22857142857142856, 'f': 0.2253521076770483}, 'rouge-l': {'r': 0.34375, 'p': 0.3235294117647059, 'f': 0.3333333283379248}}
Generating...
  1. Check if you are usi